In [10]:
# Step 1: Install required packages
print("Installing Python dependencies...")
!pip install -q ollama ragas langchain langchain-community langchain-ollama
print("Python dependencies installed.")

# Step 2: Install Ollama
print("Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh  | sh
print("Ollama installation completed.")

# Step 3: Start Ollama server in the background
import threading
import time
import subprocess

def run_ollama_serve():
    print("Starting Ollama server...")
    subprocess.run(["ollama", "serve"])

print("Spawning background thread to start Ollama server...")
threading.Thread(target=run_ollama_serve).start()
time.sleep(5)  # Wait for Ollama to initialize
print("Ollama is running in the background.")

# Step 4: Pull gemma3:1b and bge-m3 models
import ollama

print("Pulling gemma3:1b model...")
ollama.pull("gemma3:1b")
print("Successfully pulled gemma3:1b")

print("Pulling bge-m3 embedding model...")
ollama.pull("bge-m3")
print("Successfully pulled bge-m3")

# Step 5: Set up LangChain + Ragas integration
print("Setting up LangChain and Ragas components...")

import os
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.messages import HumanMessage
from ragas.llms import LangchainLLMWrapper
from ragas import evaluate
from datasets import Dataset
from ragas.metrics import answer_correctness, answer_similarity

# Dummy OpenAI key (to avoid errors)
os.environ['OPENAI_API_KEY'] = 'no-key'

# Custom wrapper to log input prompts and model outputs
class DebuggingLLM(ChatOllama):
    def invoke(self, input, *args, **kwargs):
        # Log prompt
        if isinstance(input, HumanMessage):
            prompt = input.content
        else:
            prompt = str(input)

        print("\n[DEBUG] Prompt sent to LLM:")
        print(prompt[:500] + "..." if len(prompt) > 500 else prompt)

        # Call original LLM
        response = super().invoke(input, *args, **kwargs)

        # Log response
        output_text = response.content
        print("\n[DEBUG] LLM Response:")
        print(output_text[:500] + "..." if len(output_text) > 500 else output_text)

        return response


# Initialize Embeddings
print("Initializing OllamaEmbeddings with bge-m3...")
ollama_e = OllamaEmbeddings(model="bge-m3", base_url="http://localhost:11434")
ollama_embed_self = LangchainEmbeddingsWrapper(ollama_e)
print("Embedding wrapper initialized.")

# Initialize LLM (gemma3:1b) using our debugging wrapper
print("Initializing ChatOllama with gemma3:1b (debug mode)...")
ollama_llm = DebuggingLLM(model="gemma3:1b", base_url="http://localhost:11434")
wrapper = LangchainLLMWrapper(ollama_llm)
print("LLM wrapper initialized.")

# Assign LLM and embeddings to metrics
print("Assigning LLM and embeddings to evaluation metrics...")
metrics = [answer_correctness, answer_similarity]
for m in metrics:
    print(f"Setting LLM for metric: {m}")
    m.llm = wrapper
    if hasattr(m, "embeddings"):
        print(f"Setting embeddings for metric: {m}")
        m.embeddings = ollama_embed_self
print("Metrics setup complete.")

# Example data (replace these with your real data)
questions = ["What is the capital of France?", "Explain quantum computing."]
labels = ["Paris is the capital of France.", "Quantum computing uses qubits."]
naive_rag_answers = ["Paris", "It uses qubits."]

# Create dataset
print("Creating evaluation dataset...")
dataset = Dataset.from_dict({
    "question": questions,
    "ground_truth": labels,
    "answer": naive_rag_answers,
})
print("Dataset created.")

# Evaluate
print("Starting evaluation process...")
results = evaluate(
    dataset,
    metrics=metrics,
    embeddings=ollama_embed_self,
    llm=wrapper
)
print("Evaluation completed.")

# Print results
print("\n=== Evaluation Results ===")
print(results)

Installing Python dependencies...
Python dependencies installed.
Installing Ollama...
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Ollama installation completed.
Spawning background thread to start Ollama server...
Starting Ollama server...
Ollama is running in the background.
Pulling gemma3:1b model...
Successfully pulled gemma3:1b
Pulling bge-m3 embedding model...
Successfully pulled bge-m3
Setting up LangChain and Ragas components...
Initializing OllamaEmbeddings with bge-m3...
Embedding wrapper initialized.
Initializing ChatOllama with gemma3:1b (debug mode)...
LLM wrapper initialized.
Assigni

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluation completed.

=== Evaluation Results ===
{'answer_correctness': 0.5184, 'semantic_similarity': 0.7235}
